In [60]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 13.0 MB/s eta 0:00:00
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 40.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.5 MB/s eta 0:00:00
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.4 MB/s eta 0:00:00


In [13]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import torch
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer,BertForSequenceClassification
np.set_printoptions(suppress=True)

In [36]:
extreme_left=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/ExtremeLeft.csv")
extreme_right=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/ExtremeRight.csv") 
extreme_left['label']=0
extreme_right['label']=1
extreme_left=extreme_left.loc[:,['text','label']]
extreme_right=extreme_right.loc[:,['text','label']]
extreme_right=extreme_right.sample(2014)
left_right=pd.concat([extreme_left,extreme_right])


,text,label
0,Actually George Soros was controlling the Viol...,0
1,Soros needed to be Ban from this country. Sor...,0
2,connected to Soros through OPEN SOCIETY FOUNDA...,0
3,🌐#KhazarianMafia Didn’t the Rothschild family ...,0
4,Meanwhile George Soros is on video saying he h...,0
...,...,...
1288,everything is racis thry even call me racistRT...,1
2411,Republicans like Kirk need to condemn legal im...,1
2236,>>408295812 I will not support the US. I will ...,1
946,>>17956123 That seems racis.Lemme’ cheq muh gu...,1


In [4]:
antisemitism=pd.read_csv("/Users/dhanushkikkisetti/Documents/Research Assistant/Antisemitism0.csv")
data['text']=antisemitism["text"]

NameError: name 'data' is not defined

In [14]:
import datasets
from datasets import load_dataset
dataset=load_dataset("csv", data_files='Documents/Research Assistant/Antisemitism0.csv', split=datasets.Split.TRAIN)
dataset

Using custom data configuration default-c91afebe5ab2f940
Found cached dataset csv (/Users/dhanushkikkisetti/.cache/huggingface/datasets/csv/default-c91afebe5ab2f940/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['text'],
    num_rows: 3016
})

In [15]:
from transformers import AutoModelForMaskedLM
bert_maskedML=AutoModelForMaskedLM.from_pretrained('bert-base-uncased')
from transformers import AutoTokenizer
bert_tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
tok_oup=dataset.map(lambda x:bert_tokenizer(x['text'] , padding='max_length',truncation=True,max_length=512), batched=True)

output_form=tok_oup.remove_columns('text')
output_form.set_format("torch",columns=['input_ids', 'token_type_ids', 'attention_mask'])
output_form.features
tok_oup[3]


Loading cached processed dataset at /Users/dhanushkikkisetti/.cache/huggingface/datasets/csv/default-c91afebe5ab2f940/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-8e4c2e41237ba89e.arrow


{'text': ">>409049295 >She got to live a long life, unlike the innocent little children she helped murder by her inaction.Inaction is not morally wrong. If I refuse to show a lost person the way home, I do them no wrong. But if I show them a false road, I certainly do them wrong. You have the right to remain silent, but anything you say can and will be held against you in the court of law--i.e. the legal is the inversion of the ethical. You have the psychological profile of a jew--a fanatic, intolerant, hysterical, proselytizing and vengeful crusader with a messiah complex. That is why you don't understand simple things like justice, crime, punishment, law and order. People like you should be barred from participating in any of these, because you destroy them all for everyone else.",
 'input_ids': [101,
  1028,
  1028,
  2871,
  21057,
  26224,
  24594,
  2629,
  1028,
  2016,
  2288,
  2000,
  2444,
  1037,
  2146,
  2166,
  1010,
  4406,
  1996,
  7036,
  2210,
  2336,
  2016,
  3271

In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15
)


In [119]:
training_args=TrainingArguments(
    output_dir="output",
    num_train_epochs=1
)
trainer = Trainer(
    bert_maskedML,
    training_args,
    data_collator=data_collator,
    train_dataset=output_form
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [120]:
trainer.train()

***** Running training *****
  Num examples = 3016
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 377
  Number of trainable parameters = 109514298


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=377, training_loss=2.8066895178837865, metrics={'train_runtime': 5986.8281, 'train_samples_per_second': 0.504, 'train_steps_per_second': 0.063, 'total_flos': 793825715404800.0, 'train_loss': 2.8066895178837865, 'epoch': 1.0})

In [121]:
trainer.save_model("/Users/dhanushkikkisetti/Documents/Research Assistant/FineTuneMaskedLMModel")

Saving model checkpoint to /Users/dhanushkikkisetti/Documents/Research Assistant/FineTuneMaskedLMModel
Configuration saved in /Users/dhanushkikkisetti/Documents/Research Assistant/FineTuneMaskedLMModel/config.json
Model weights saved in /Users/dhanushkikkisetti/Documents/Research Assistant/FineTuneMaskedLMModel/pytorch_model.bin


In [18]:
from transformers import AutoModel
bertMasked_New=AutoModel.from_pretrained("/Users/dhanushkikkisetti/Documents/Research Assistant/FineTuneMaskedLMModel")
input=["Soros needed to be Ban from this country.  Sorry nazi",
       "Yeah. Kike musuems made by kikes so that no one would forget the 6 gorillion"]
inputs=bert_tokenizer(input,padding='max_length',truncation=True,max_length=512,return_tensors='pt')
maskedLM_output=bertMasked_New(**inputs)
Masked_auto_pooler=maskedLM_output['pooler_output']


Some weights of the model checkpoint at /Users/dhanushkikkisetti/Documents/Research Assistant/FineTuneMaskedLMModel were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /Users/dhanushkikkisetti/Documents/Research Assis

In [20]:
Masked_auto_pooler.detach().numpy()

array([[-0.41872808,  0.12335812,  0.1132789 , ...,  0.0608442 ,
        -0.27820548, -0.30767083],
       [-0.38236293,  0.36598867,  0.01556826, ..., -0.20580238,
        -0.22272873, -0.24919914]], dtype=float32)

In [164]:
print(Masked_auto_pooler.shape)
Masked_auto_pooler

torch.Size([2, 768])


tensor([[ 0.5538,  0.1830, -0.0172,  ...,  0.0795,  0.1320, -0.0380],
        [ 0.5340,  0.0532, -0.0137,  ...,  0.0613,  0.2614,  0.1712]],
       grad_fn=<TanhBackward0>)

In [140]:
bert_output=AutoModel.from_pretrained("bert-base-uncased")

loading configuration file config.json from cache at /Users/dhanushkikkisetti/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /Users/dhanushkikkisetti/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b61

In [165]:

from torch.nn import CosineSimilarity
maskedLM_output=bertMasked_New(**inputs)
normalbert_output=bert_output(**inputs)
Masked_auto_pooler=maskedLM_output['pooler_output']
bert_auto_pooler=normalbert_output['pooler_output']
cos=CosineSimilarity(dim=0,eps=1e-6)
print(cos(Masked_auto_pooler[0],bert_auto_pooler[0]))
cos(Masked_auto_pooler[1],bert_auto_pooler[1])

tensor(-0.0397, grad_fn=<SumBackward1>)


tensor(-0.0417, grad_fn=<SumBackward1>)